In [1]:
# SSL连接错误解决方案
import ssl
import urllib3
import os

# 方案1：禁用SSL验证（不推荐用于生产环境）
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
os.environ['PYTHONHTTPSVERIFY'] = '0'

# 方案2：设置SSL上下文
ssl_context = ssl.create_default_context()
ssl_context.check_hostname = False
ssl_context.verify_mode = ssl.CERT_NONE

print("SSL配置已设置")


SSL配置已设置


In [2]:
import getpass
import os

if not os.getenv("DASHSCOPE_API_KEY"):
    os.environ["DASHSCOPE_API_KEY"] = getpass.getpass("Enter your Dashscope API key: ")


In [3]:
# 导入必要的模块
from langchain_community.chat_models.tongyi import ChatTongyi  # 导入通义千问聊天模型
from langchain_core.tools import tool  # 导入工具装饰器


# 使用@tool装饰器定义一个工具函数
@tool
def multiply(first_int: int, second_int: int) -> int:
    """
    将两个整数相乘的工具函数
    
    Args:
        first_int: 第一个整数
        second_int: 第二个整数
    
    Returns:
        int: 两个整数的乘积
    """
    return first_int * second_int


# 创建通义千问语言模型实例
llm = ChatTongyi(model="qwen-turbo")

# 将工具绑定到语言模型上，使模型能够调用这个工具
llm_with_tools = llm.bind_tools([multiply])

# 向模型发送一个需要计算的问题
msg = llm_with_tools.invoke("What's 5 times forty two")

# 打印模型的响应（应该包含工具调用信息）
print(msg)

content='' additional_kwargs={'tool_calls': [{'function': {'name': 'multiply', 'arguments': '{"first_int": 5, "second_int": 42}'}, 'index': 0, 'id': 'call_c9e470066e7542ab8bf6d9', 'type': 'function'}]} response_metadata={'model_name': 'qwen-turbo', 'finish_reason': 'tool_calls', 'request_id': '323283c8-d850-4231-9e30-9cdba6f75d30', 'token_usage': {'input_tokens': 213, 'output_tokens': 27, 'total_tokens': 240, 'prompt_tokens_details': {'cached_tokens': 0}}} id='run--d309eae0-cf7a-462f-891a-aef9e27c2434-0' tool_calls=[{'name': 'multiply', 'args': {'first_int': 5, 'second_int': 42}, 'id': 'call_c9e470066e7542ab8bf6d9', 'type': 'tool_call'}]


# 工具调用功能说明

## 什么是工具调用？
工具调用（Tool Calling）是让AI模型能够调用外部函数或工具的能力。当模型遇到需要计算、查询数据或执行特定操作时，它可以自动调用相应的工具。

## 代码执行流程：
1. **定义工具**：使用 `@tool` 装饰器定义 `multiply` 函数
2. **绑定工具**：使用 `bind_tools()` 将工具绑定到语言模型
3. **发送请求**：向模型发送需要计算的问题
4. **自动调用**：模型识别需要计算，自动调用 `multiply` 工具
5. **返回结果**：模型返回工具调用的结果

## 输出解释：
从输出可以看到：
- `tool_calls`: 包含工具调用的详细信息
- `function.name`: 调用的函数名 "multiply"
- `function.arguments`: 传入的参数 `{"first_int": 5, "second_int": 42}`
- `finish_reason`: "tool_calls" 表示模型决定调用工具

## 实际应用场景：
- 数学计算
- 数据查询
- API调用
- 文件操作
- 数据库查询


In [4]:
# 执行工具调用并获取最终结果
from langchain_core.messages import ToolMessage

# 检查是否有工具调用
if msg.tool_calls:
    # 获取工具调用信息
    tool_call = msg.tool_calls[0]
    print(f"工具名称: {tool_call['name']}")
    print(f"调用参数: {tool_call['args']}")
    
    # 执行工具调用
    result = multiply.invoke(tool_call['args'])
    print(f"计算结果: {result}")
    
    # 创建工具消息
    tool_message = ToolMessage(
        content=str(result),
        tool_call_id=tool_call['id']
    )
    
    # 将工具结果发送回模型，获取最终回答
    final_response = llm_with_tools.invoke([
        {"role": "user", "content": "What's 5 times forty two"},
        msg,  # 模型的工具调用
        tool_message  # 工具执行结果
    ])
    
    print(f"\n最终回答: {final_response.content}")
else:
    print("没有工具调用")


工具名称: multiply
调用参数: {'first_int': 5, 'second_int': 42}
计算结果: 210

最终回答: 5 times forty two is 210.


In [5]:
# 导入必要的模块
from langchain_community.chat_models.tongyi import ChatTongyi  # 导入通义千问聊天模型
from langchain_core.messages import HumanMessage, SystemMessage  # 导入消息类型

# 定义工具列表 - 使用字典格式定义工具（OpenAI Function Calling 格式）
tools = [
    {
        "type": "function",  # 工具类型：函数
        "function": {
            "name": "get_current_time",  # 函数名称
            "description": "当你想知道现在的时间时非常有用。",  # 函数描述
            "parameters": {},  # 函数参数（无参数）
        },
    },
    {
        "type": "function",  # 工具类型：函数
        "function": {
            "name": "get_current_weather",  # 函数名称
            "description": "当你想查询指定城市的天气时非常有用。",  # 函数描述
            "parameters": {  # 函数参数定义
                "type": "object",  # 参数类型：对象
                "properties": {  # 参数属性
                    "location": {  # 位置参数
                        "type": "string",  # 参数类型：字符串
                        "description": "城市或县区，比如北京市、杭州市、余杭区等。",  # 参数描述
                    }
                },
            },
            "required": ["location"],  # 必需参数列表
        },
    },
]

# 构建对话消息列表
messages = [
    SystemMessage(content="You are a Tools helpful assistant."),  # 系统消息：定义AI角色
    HumanMessage(content="What is the weather like in San Francisco?"),  # 用户消息：询问天气
]

# 创建通义千问聊天模型实例
chatLLM = ChatTongyi()

# 定义模型调用参数
llm_kwargs = {
    "tools": tools,  # 绑定工具列表
    "result_format": "message"  # 结果格式：消息格式
}

# 绑定工具并调用模型
ai_message = chatLLM.bind(**llm_kwargs).invoke(messages)

# 显示模型响应
ai_message

AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'name': 'get_current_weather', 'arguments': '{"location": "San Francisco"}'}, 'index': 0, 'id': 'call_27ae3606c3c74c74a3c698', 'type': 'function'}]}, response_metadata={'model_name': 'qwen-turbo', 'finish_reason': 'tool_calls', 'request_id': '98a5d548-d2ab-414b-aea1-b5c23182b292', 'token_usage': {'input_tokens': 221, 'output_tokens': 21, 'total_tokens': 242, 'prompt_tokens_details': {'cached_tokens': 0}}}, id='run--4a58ead2-da1d-45df-94b5-1c66028f8bab-0', tool_calls=[{'name': 'get_current_weather', 'args': {'location': 'San Francisco'}, 'id': 'call_27ae3606c3c74c74a3c698', 'type': 'tool_call'}])

# 工具定义的两种方式对比

## 方式一：使用 @tool 装饰器（推荐）
```python
@tool
def multiply(first_int: int, second_int: int) -> int:
    """将两个整数相乘的工具函数"""
    return first_int * second_int

llm_with_tools = llm.bind_tools([multiply])
```

## 方式二：使用字典格式（OpenAI Function Calling 格式）
```python
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "当你想查询指定城市的天气时非常有用。",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "城市或县区，比如北京市、杭州市、余杭区等。",
                    }
                },
                "required": ["location"],
            },
        },
    }
]

llm_kwargs = {"tools": tools, "result_format": "message"}
ai_message = chatLLM.bind(**llm_kwargs).invoke(messages)
```

## 两种方式的区别：

### @tool 装饰器方式：
- ✅ **简单易用**：直接定义Python函数
- ✅ **类型安全**：自动类型检查
- ✅ **自动生成**：自动生成工具描述和参数
- ✅ **可执行**：函数可以直接调用

### 字典格式方式：
- ✅ **标准化**：符合OpenAI Function Calling规范
- ✅ **灵活性**：可以定义虚拟工具（不需要实际函数）
- ✅ **兼容性**：与更多模型兼容
- ❌ **复杂**：需要手动编写JSON Schema
- ❌ **不可执行**：只是工具描述，需要额外实现

## 使用建议：
- **实际功能**：使用 `@tool` 装饰器
- **虚拟工具**：使用字典格式
- **复杂参数**：使用字典格式更灵活


In [ ]:
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_core.messages import AIMessage, HumanMessage

messages = [
    HumanMessage(
        content="""请续写句子"春天来了，大地"来表达春天的美丽和作者的喜悦之情。"""
    ),
    AIMessage(
        content="春天来了，大地", additional_kwargs={"partial": True}
    ),
]
chatLLM = ChatTongyi()
ai_message = chatLLM.invoke(messages)
ai_message

AIMessage(content='苏醒，万物复苏，花儿竞相开放，绿草如茵，空气中弥漫着淡淡的花香。鸟儿在枝头欢唱，仿佛在赞美这美好的季节。我漫步在田野间，心中充满喜悦，感受到春天的温暖与生机，仿佛整个世界都焕然一新，充满了希望与活力。', additional_kwargs={}, response_metadata={'model_name': 'qwen-turbo', 'finish_reason': 'stop', 'request_id': '73bbb2c1-ae97-4a73-a6e2-49e92cc9be08', 'token_usage': {'input_tokens': 37, 'output_tokens': 70, 'total_tokens': 107, 'prompt_tokens_details': {'cached_tokens': 0}}}, id='run--4b621655-1762-485c-8181-31de1bf47ab8-0')